In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Loading data
img = cv2.imread('./testy/000.jpg')
plt.imshow(img[...,::-1])
plt.show()

In [ ]:
# Board detection
def get_contour(img):
    grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey,(7,7),0)
    edges = cv2.Canny(blur, 50, 100, apertureSize = 3)
    kernel = np.ones((5,5), np.uint8)
    dilation = cv2.dilate(edges,kernel,iterations = 2)
    contour_img, contours, hierarchy = cv2.findContours(
                                            dilation, 
                                            cv2.RETR_TREE,
                                            cv2.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv2.contourArea)
    contour_img = img.copy()
    contour_img = cv2.drawContours(contour_img, [cnt], 0, (0,255,0), 3)
    return contour_img, cnt


def get_points(cnt):
    epsilon = 0.01*cv2.arcLength(cnt,True)
    approx = cv2.approxPolyDP(cnt,epsilon,True)
    x_center = np.mean(approx[:,:,0])
    y_center = np.mean(approx[:,:,1])

    #sorting
    pts = np.zeros((4,2))
    for x in approx:
        for X, Y in x:
            if X < x_center and Y < y_center:
                pts[0,0] = X
                pts[0,1] = Y
            if X < x_center and Y > y_center:
                pts[1,0] = X
                pts[1,1] = Y
            if X > x_center and Y > y_center:
                pts[2,0] = X
                pts[2,1] = Y
            if X > x_center and Y < y_center:
                pts[3,0] = X
                pts[3,1] = Y
    size = max(approx.flatten()) - min(approx.flatten())
    return pts, size


def warp_image(img, pts, size):
    pts = np.float32(pts)
    pts_sqr = np.float32([[0,0],[0,size],[size,size],[size,0]])
    M = cv2.getPerspectiveTransform(pts,pts_sqr)
    warped_img = cv2.warpPerspective(img,M,(size,size))
    return warped_img

def warp_image(img, pts, size):
    pts = np.float32(pts)
    pts_sqr = np.float32([[0,0],[0,size],[size,size],[size,0]])
    M = cv2.getPerspectiveTransform(pts,pts_sqr)
    warped_img = cv2.warpPerspective(img,M,(size,size))
    return warped_img

def slice_image(image):
    slices = list()
    samples_x = np.linspace(0,image.shape[0],16, dtype=np.int)
    samples_y = np.linspace(0,image.shape[1],16,dtype=np.int)
    for x_start, x_end in zip(samples_x, samples_x[1:]):
        for y_start, y_end in zip(samples_y, samples_y[1:]):
            slices.append(image[x_start:x_end,y_start:y_end])
    return slices

In [ ]:
contour_img, cnt = get_contour(img)
pts, size = get_points(cnt)
board = warp_image(img, pts, size)
warped_img = board[...,::-1]
plt.imshow(warped_img)
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from skimage.transform import resize
from skimage import color
from skimage import io

In [ ]:
# Tile classifier setup, training
tile_classifier = Sequential()
tile_classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
tile_classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
tile_classifier.add(MaxPooling2D(pool_size = (2, 2)))
tile_classifier.add(Dropout(0.15))
tile_classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
tile_classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
tile_classifier.add(MaxPooling2D(pool_size = (2, 2)))
tile_classifier.add(Flatten())
tile_classifier.add(Dense(units = 128, activation = 'relu'))
tile_classifier.add(Dense(units = 2, activation = 'softmax'))
tile_classifier.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
datagen = ImageDataGenerator(rescale=1. / 255)
itr_tiles =  datagen.flow_from_directory(
            './data_set_tile',
            target_size=(64,64),
            batch_size= 1868,
            class_mode='categorical')
X, y = itr_tiles.next()
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=7)

# tile_classifier.fit(X_train, y_train, epochs=5, batch_size=X_train.shape[0]//20,verbose = 1)
# tile_classifier.save_weights('tile_detector.h5') 
tile_classifier.load_weights('tile_detector.h5')
# tile_classifier.evaluate(X_test,y_test)

In [ ]:
letter_classifier = Sequential()

letter_classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
letter_classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
letter_classifier.add(MaxPooling2D(pool_size = (2, 2)))
letter_classifier.add(Dropout(0.15))

letter_classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
letter_classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
letter_classifier.add(MaxPooling2D(pool_size = (2, 2)))
letter_classifier.add(Dropout(0.15))

letter_classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
letter_classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
letter_classifier.add(MaxPooling2D(pool_size = (2, 2)))

letter_classifier.add(Flatten())
letter_classifier.add(Dense(units = 128, activation = 'relu'))
letter_classifier.add(Dense(units = 33, activation = 'softmax'))

letter_classifier.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
datagen = ImageDataGenerator(
                rescale=1. / 255, 
                rotation_range=10,
                width_shift_range = 0.1,
                height_shift_range = 0.1)
itr_letters =  datagen.flow_from_directory(
            './data_set_letter',
            target_size=(64,64),
            batch_size= 812,
            class_mode='categorical')

X, y = itr_letters.next()
for i in range(5):
    X_temp,y_temp = itr_letters.next()
    X = np.vstack((X,X_temp))
    y = np.vstack((y,y_temp))

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=7)

# letter_classifier.fit(X_train, y_train, epochs=20, batch_size=X_train.shape[0]//20,verbose = 1)
# letter_classifier.save_weights('letters_classifier.h5') 
letter_classifier.load_weights('letters_classifier.h5')
letter_classifier.evaluate(X_test,y_test)

In [ ]:
from skimage.transform import resize
slices = slice_image(warped_img)
predictions = list()
for slice_ in slices:
    scaled_img = resize(slice_,(64,64,3),mode='reflect')
    img = np.expand_dims(scaled_img, axis = 0)
    tile_prediction = tile_classifier.predict(img)
    class_prediction = np.argmax(tile_prediction.flatten()) - 1
    if(class_prediction == 0):
        letter_prediction = letter_classifier.predict(img)
        class_prediction = np.argmax(letter_prediction)
    predictions.append(class_prediction)
    
predictions_board = np.array(predictions).reshape(15,15)
print(predictions_board)
# tile_indexes = [i for i, x in enumerate(indexes) if x == 1]
# letter_slices = np.array(slices)[tile_indexes]

In [ ]:
mapping = itr_letters.class_indices
mapping.update({'_':-1,
                'x':0})
inverse_mapping = {v:k[:1] for k, v in mapping.items()}
mapped_board = np.vectorize(inverse_mapping.get)(predictions_board)
print(mapped_board)


In [ ]:
letter_indexes = []
for letter_slice in letter_slices:
    scaled_img = resize(letter_slice,(64,64,3),mode='reflect')
    img = np.expand_dims(scaled_img, axis = 0)
    prediction = letter_classifier.predict(img)
    letter_indexes.append(np.argmax(prediction.flatten()))

In [ ]:
fig = plt.figure(figsize=(5, 5),dpi= 100)
plt.imshow(predictions_board)
plt.grid(True)
plt.show()